In [26]:
#1. import libraries
import pandas as pd
from sqlalchemy import create_engine, text
import json
import psycopg2
import matplotlib.pyplot as plt

In [27]:
#2. Load the database configuration
with open (r"C:\Users\user\Documents\Portfolio_Vista\Retail_Fashion\scripts\db_config.json","r") as file:
    config = json.load(file)

username = config["username"]
password = config["password"]
host = config["host"]
port = config["port"]
database = config["database"]

print ('Succesful loaded the credential!')

Succesful loaded the credential!


In [28]:
#3. Connect to PostgreSQL
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
print (f'Connected to database {database}')

Connected to database shoes


In [29]:
#4. Query
#1. Which countries are the largest markets by quantity and revenue?
query = """
select
	country,
	sum (amount)::numeric(16,2) as revenue,
	sum (quantity) as total_sold
from 
	sales_shoes
group by
	country
order by 
	revenue desc,
	total_sold desc;
"""
df_1 = pd.read_sql(query, engine)
display (df_1)

,country,revenue,total_sold
0,Japan,31587.39,170.0
1,Canada,30851.19,174.0
2,Germany,29822.89,155.0
3,UK,29739.25,191.0
4,USA,26095.59,138.0
5,Australia,24094.02,144.0
6,India,20912.74,125.0


In [30]:
#2. How does gender affect product preference?
query = """
select 
	gender,
	product_type,
	sum(quantity) as total_sold
from 
	sales_shoes
group by
	gender, product_type 
order by
	gender, total_sold desc;
"""
df_2 = pd.read_sql(query, engine)
display (df_2)

,gender,product_type,total_sold
0,Men,Sneakers,162.0
1,Men,Hoodie,157.0
2,Men,Cap,35.0
3,Men,T-shirt,29.0
4,Men,Joggers,25.0
5,Unisex,Sneakers,107.0
6,Unisex,Hoodie,88.0
7,Unisex,Joggers,33.0
8,Unisex,T-shirt,30.0
9,Unisex,Cap,14.0


In [31]:
#3. Which categories are most popular in each region?
query = """
with rank_category as (
	select
		country,
		category,
		sum (quantity) as total_sold,
		row_number() over (partition by country order by sum (quantity) desc) as rank_category_country
	from 
		sales_shoes
	group by
		country, category
	)
select
	country,
	category,
	total_sold
from 
	rank_category
where 
	rank_category_country = 1
order by
	total_sold desc;
"""
df_3 = pd.read_sql(query, engine)
display (df_3)

,country,category,total_sold
0,Canada,Streetwear,69.0
1,UK,Casual,62.0
2,Germany,Casual,57.0
3,Japan,Limited Edition,52.0
4,USA,Streetwear,41.0
5,Australia,Limited Edition,40.0
6,India,Limited Edition,35.0


In [32]:
#4. Are certain brands more popular among men or women?
query = """
with men_preference as(
	select
		brand,
		gender,
		sum(quantity) as total_sold,
		row_number () over (partition by gender order by sum(quantity) desc) as rank_brand
	from 
		sales_shoes
	where 
		gender = 'Men'
	group by
		gender, brand
	),
women_preference as (
	select
		brand,
		gender,
		sum(quantity) as total_sold,
		row_number () over (partition by gender order by sum(quantity) desc) as rank_brand
	from 
		sales_shoes
	where 
		gender = 'Women'
	group by
		gender, brand 
	)
select 
	 m.total_sold, m.rank_brand as men_rank,m.brand,
	 w.rank_brand as women_rank, w.total_sold 
from
	men_preference m
left join
	women_preference w
using (brand);
"""
df_4 = pd.read_sql(query, engine)
display (df_4)

,total_sold,men_rank,brand,women_rank,total_sold
0,131.0,1,Nike,1,148.0
1,82.0,2,Adidas,2,71.0
2,60.0,3,Off-White,4,51.0
3,46.0,4,Supreme,6,29.0
4,35.0,5,New Era,5,39.0
5,29.0,6,Essentials,7,21.0
6,25.0,7,Puma,3,58.0


In [33]:
#5. Which payment modes are most frequently used?
query = """
select
	payment_mode,
	count (payment_mode) as total_transaction
from 
	sales_shoes
group by
	payment_mode
order by 
	total_transaction desc
limit 1;
"""
df_5 = pd.read_sql(query, engine)
display (df_5)

,payment_mode,total_transaction
0,Cash on Delivery,102


In [34]:
data_table = [df_1, df_2, df_3, df_4, df_5]
name_table = [1,2,3,4,5]
titles = [
    "Top Markets by Sales Quantity and Revenue",
    "Product Preferences by Gender",
    "Category Popularity by Region",
    "Brand and Product Popularity by Gender",
    "Most Frequently Used Payment Methods"
]
dir_img = r'C:\Users\user\Documents\Portfolio_Vista\Retail_Fashion\data\market_insight_query'
for i,j,k in zip (data_table,name_table,titles):
    fig, ax = plt.subplots(figsize=(len(i.columns) * 3, len(i) * 0.7))
    ax.axis('tight')
    ax.axis('off')

    table = ax.table(
        cellText=i.values,
        colLabels=i.columns,
        loc='center',
        cellLoc='center'
    )
    table.scale(1, 1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(10)

    plt.title(k, fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(f"{dir_img}\market_insight_{j}.png", bbox_inches='tight', dpi=300)
    plt.close()


<>:28: SyntaxWarning: invalid escape sequence '\m'
<>:28: SyntaxWarning: invalid escape sequence '\m'
C:\Users\user\AppData\Local\Temp\ipykernel_1344\2958050342.py:28: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(f"{dir_img}\market_insight_{j}.png", bbox_inches='tight', dpi=300)
C:\Users\user\AppData\Local\Temp\ipykernel_1344\2958050342.py:27: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
